In [1]:
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import numpy as np
nltk.download("punkt", quiet=True)

c:\Users\ungdu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [2]:
file_path = r'C:\Users\ungdu\Downloads\Test_Chatgpt\test_data.csv' #import data
data = pd.read_csv(file_path)
data

,Câu hỏi,Câu trả lời
0,Các quả có mùi vị như thế nào?,Quả cam ngon. Quả táo dở. Quả chanh chua. Quả ...
1,Các quả có hình dáng như thế nào?,"Quả cam có hình tròn. Quả táo có hình tròn, hơ..."


In [3]:
import pandas as pd
import nltk
from IPython.display import display

# Tải bộ dữ liệu phân tách câu của NLTK nếu chưa có
nltk.download('punkt')


# Danh sách chứa tất cả các câu đã phân tách
all_sentences = []

# Lặp qua từng dòng của DataFrame
for _, row in data.iterrows():
    # Lấy nội dung cột "Câu trả lời"
    text = row.get("Câu trả lời", "")
    
    # Kiểm tra nếu text không rỗng và là chuỗi
    if text and isinstance(text, str):
        # Chia văn bản thành các câu
        sentences = nltk.sent_tokenize(text)
        
        # Hiển thị từng câu đã tách (chỉ để kiểm tra)
        display(sentences)
        
        # Thêm các câu này vào danh sách tổng hợp
        all_sentences.extend(sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ungdu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Quả cam ngon.',
 'Quả táo dở.',
 'Quả chanh chua.',
 'Quả mít to.',
 'Quả mít rất thơm nữa.']

['Quả cam có hình tròn.',
 'Quả táo có hình tròn, hơi nhỏ.',
 'Quả chanh hình bầu dục.',
 'Quả mít to dài có vỏ xù xì.',
 'Quả mít có thể lấy gỗ.']

In [4]:
# Tạo DataFrame từ danh sách các câu
sentences_df = pd.DataFrame({'Câu trả lời': all_sentences})

# Lưu DataFrame vào file CSV
output_filename = 'extracted_sentences.csv'
sentences_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu {len(all_sentences)} câu vào file '{output_filename}'.")

Đã lưu 10 câu vào file 'extracted_sentences.csv'.


In [5]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Đọc file CSV chứa các câu đã được tách
sentences_df = pd.read_csv('extracted_sentences.csv', encoding='utf-8-sig')
print(sentences_df)
# Hiển thị số lượng câu đã được tải
print(f"Đã tải {len(sentences_df)} câu từ 'extracted_sentences.csv'.")

                      Câu trả lời
0                   Quả cam ngon.
1                     Quả táo dở.
2                 Quả chanh chua.
3                     Quả mít to.
4           Quả mít rất thơm nữa.
5           Quả cam có hình tròn.
6  Quả táo có hình tròn, hơi nhỏ.
7         Quả chanh hình bầu dục.
8     Quả mít to dài có vỏ xù xì.
9          Quả mít có thể lấy gỗ.
Đã tải 10 câu từ 'extracted_sentences.csv'.


In [6]:
# Bước 3: Vector hóa các câu sử dụng TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences_df['Câu trả lời'])

# Hiển thị kích thước ma trận TF-IDF
print(f"Ma trận TF-IDF có kích thước: {tfidf_matrix.shape}")

# Bước 2: Tính độ tương đồng giữa các câu bằng cosine similarity
# Tính ma trận độ tương đồng giữa các câu
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Hiển thị một phần ma trận độ tương đồng
print("Mẫu độ tương đồng giữa các câu:")
print(cosine_sim[:5, :5])

Ma trận TF-IDF có kích thước: (10, 26)
Mẫu độ tương đồng giữa các câu:
[[1.         0.07351663 0.07351663 0.08803749 0.0530266 ]
 [0.07351663 1.         0.07351663 0.08803749 0.0530266 ]
 [0.07351663 0.07351663 1.         0.08803749 0.0530266 ]
 [0.08803749 0.08803749 0.08803749 1.         0.26661112]
 [0.0530266  0.0530266  0.0530266  0.26661112 1.        ]]


In [7]:
# 1. Lưu Ma Trận TF-IDF
# Chuyển đổi ma trận TF-IDF thành DataFrame
# Lưu ý: Nếu số lượng từ vựng (features) lớn, việc lưu trữ có thể tiêu tốn nhiều dung lượng
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Lưu DataFrame TF-IDF vào CSV
tfidf_output_filename = 'tfidf_matrix.csv'
tfidf_df.to_csv(tfidf_output_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu ma trận TF-IDF vào file '{tfidf_output_filename}'.")

Đã lưu ma trận TF-IDF vào file 'tfidf_matrix.csv'.


In [8]:
# 2. Lưu Ma Trận Cosine Similarity
# Chuyển đổi ma trận cosine similarity thành DataFrame
cosine_sim_df = pd.DataFrame(cosine_sim, index=sentences_df.index, columns=sentences_df.index)

# Lưu DataFrame cosine similarity vào CSV
cosine_sim_output_filename = 'cosine_similarity_matrix.csv'
cosine_sim_df.to_csv(cosine_sim_output_filename, index=True, encoding='utf-8-sig')

print(f"Đã lưu ma trận cosine similarity vào file '{cosine_sim_output_filename}'.")

Đã lưu ma trận cosine similarity vào file 'cosine_similarity_matrix.csv'.


In [9]:
# Bước 4: Gép các câu lại với nhau dựa trên độ tương đồng

# Thiết lập ngưỡng độ tương đồng
SIMILARITY_THRESHOLD = 0.3  # Bạn có thể điều chỉnh ngưỡng này

# Danh sách để lưu các chunk
chunks = []
current_chunk = sentences_df.iloc[0]['Câu trả lời']  # Bắt đầu với câu đầu tiên

for i in range(1, len(sentences_df)):
    prev_sentence = sentences_df.iloc[i - 1]['Câu trả lời']
    current_sentence = sentences_df.iloc[i]['Câu trả lời']
    
    # Vector hóa câu hiện tại và câu trước đó
    prev_vector = vectorizer.transform([prev_sentence])
    current_vector = vectorizer.transform([current_sentence])
    
    # Tính độ tương đồng giữa câu trước và câu hiện tại
    similarity = cosine_similarity(prev_vector, current_vector)[0][0]
    
    print(f"Độ tương đồng giữa câu {i} và câu {i+1}: {similarity:.2f}")
    
    if similarity >= SIMILARITY_THRESHOLD:
        # Gép câu hiện tại vào chunk hiện tại
        current_chunk += ' ' + current_sentence
    else:
        # Thêm chunk hiện tại vào danh sách và bắt đầu chunk mới
        chunks.append(current_chunk)
        current_chunk = current_sentence

# Thêm chunk cuối cùng vào danh sách
chunks.append(current_chunk)

# Tạo DataFrame từ các chunk
chunks_df = pd.DataFrame({'Chunk': chunks})

# Lưu các chunk vào file CSV mới
output_chunk_filename = 'chunked_sentences.csv'
chunks_df.to_csv(output_chunk_filename, index=False, encoding='utf-8-sig')

print(f"Đã lưu {len(chunks)} chunks vào file '{output_chunk_filename}'.")

Độ tương đồng giữa câu 1 và câu 2: 0.07
Độ tương đồng giữa câu 2 và câu 3: 0.07
Độ tương đồng giữa câu 3 và câu 4: 0.09
Độ tương đồng giữa câu 4 và câu 5: 0.27
Độ tương đồng giữa câu 5 và câu 6: 0.05
Độ tương đồng giữa câu 6 và câu 7: 0.54
Độ tương đồng giữa câu 7 và câu 8: 0.17
Độ tương đồng giữa câu 8 và câu 9: 0.03
Độ tương đồng giữa câu 9 và câu 10: 0.21
Đã lưu 9 chunks vào file 'chunked_sentences.csv'.


In [10]:
# --- Bước 7: Thêm Cột "Chunk" vào Dữ Liệu Gốc ---

# Khởi tạo cột 'Chunk' trong sentences_df
sentences_df['Chunk'] = None

# Duyệt qua từng chunk và gán cho các câu thuộc chunk đó
for idx, row in chunks_df.iterrows():
    chunk_text = row['Chunk']
    # Sử dụng NLTK để phân tách các câu trong chunk
    chunk_sentences = nltk.sent_tokenize(chunk_text)
    
    for sent in chunk_sentences:
        # Tìm chỉ số của câu trong sentences_df
        match_idx = sentences_df[sentences_df['Câu trả lời'] == sent].index
        if not match_idx.empty:
            sentences_df.at[match_idx[0], 'Chunk'] = chunk_text
        else:
            print(f"Warning: Sentence '{sent}' not found in sentences_df.")

# Kiểm tra các chunk đã được gán
print("\nDữ liệu với cột 'Chunk':")
print(sentences_df)

# --- Bước 8: Gộp Cột "Chunk" Vào Dữ Liệu Gốc ---
# Merge dựa trên cột 'Câu trả lời'
data_with_chunks = pd.merge(data, sentences_df[['Câu trả lời', 'Chunk']], on='Câu trả lời', how='left')

# Hiển thị dữ liệu đã được gán 'Chunk'
print("\nDữ liệu gốc sau khi thêm cột 'Chunk':")
print(data_with_chunks)

# --- Bước 9: Lưu Dữ Liệu Với Cột "Chunk" ---
output_data_with_chunk = 'test_data_with_chunk.csv'
data_with_chunks.to_csv(output_data_with_chunk, index=False, encoding='utf-8-sig')
print(f"\nĐã lưu dữ liệu với cột 'Chunk' vào file '{output_data_with_chunk}'.")


Dữ liệu với cột 'Chunk':
                      Câu trả lời  \
0                   Quả cam ngon.   
1                     Quả táo dở.   
2                 Quả chanh chua.   
3                     Quả mít to.   
4           Quả mít rất thơm nữa.   
5           Quả cam có hình tròn.   
6  Quả táo có hình tròn, hơi nhỏ.   
7         Quả chanh hình bầu dục.   
8     Quả mít to dài có vỏ xù xì.   
9          Quả mít có thể lấy gỗ.   

                                               Chunk  
0                                      Quả cam ngon.  
1                                        Quả táo dở.  
2                                    Quả chanh chua.  
3                                        Quả mít to.  
4                              Quả mít rất thơm nữa.  
5  Quả cam có hình tròn. Quả táo có hình tròn, hơ...  
6  Quả cam có hình tròn. Quả táo có hình tròn, hơ...  
7                            Quả chanh hình bầu dục.  
8                        Quả mít to dài có vỏ xù xì.  
9               

In [11]:
import math

def divide_dataframe(data_with_chunks, batch_size):
    num_batches = math.ceil(len(data_with_chunks) / batch_size)  # Tính số lượng batch
    return [data_with_chunks.iloc[i * batch_size:(i + 1) * batch_size] for i in range(num_batches)] # Định nghĩa khoảng cho từng batch

In [12]:
import uuid

def process_batch(batch_df, model, collection):
    """Mã hóa và lưu dữ liệu vào Chroma vector store cho batch này."""
    try:
        # Mã hóa dữ liệu trong cột 'chunk' thành vector cho batch này
        embeddings = model.encode(batch_df['Chunk'].tolist())

        # Thu thập tất cả metadata vào một danh sách
        metadatas = [row.to_dict() for _, row in batch_df.iterrows()]

        # Tạo ID duy nhất cho mỗi phần tử trong batch
        batch_ids = [str(uuid.uuid4()) for _ in range(len(batch_df))]

        # Thêm batch vào Chroma collection
        collection.add(
            ids=batch_ids,
            embeddings=embeddings,
            metadatas=metadatas
        )

    except Exception as e:
        print(f"Xảy ra lỗi khi thêm dữ liệu vào Chroma: {str(e)}")

In [ ]:
import chromadb

chroma_client = chromadb.Client()
model = SentenceTransformer('keepitreal/vietnamese-sbert')
batch_size = 16

# Chia DataFrame thành các batch nhỏ
df_batches = divide_dataframe(chunks_df, batch_size)

# Kiểm tra nếu collection đã tồn tại hoặc tạo mới
collection_name = "my_collection1"
collection = chroma_client.get_or_create_collection(name=collection_name)

# In ra thông tin collection để xác nhận
print(f"Collection '{collection_name}' đã được tạo hoặc lấy thành công.")

Collection 'my_collection1' đã được tạo hoặc lấy thành công.


: 

In [ ]:
# Xử lý từng batch và thêm vào collection
for i, batch_df in enumerate(df_batches):
    if batch_df.empty:
        continue  # Bỏ qua batch trống
    process_batch(batch_df, model, collection)

# Kiểm tra và in ra số lượng items đã được lưu vào collection
result = collection.get(include=["metadatas", "embeddings"])  # Lấy dữ liệu từ collection
print(f"Số lượng phần tử trong collection: {len(result['metadatas'])}")

In [ ]:
for i in range(min(100, len(result['metadatas']))):  # Hiển thị tối đa 5 phần tử
    print(f"Metadata {i+1}: {result['metadatas'][i]}")  # In ra thông tin metadata của phần tử thứ i